In [2]:
from database.queries import fetch_table, fetch_count

2025-08-13 17:05:36.591 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-08-13 17:05:36.592 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
count = fetch_count("analysis_results")
count

2025-08-13 17:05:36.596 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
/Users/nike/Documents/Data Science Work/Project/Hackthon/CX_Management_project/cx_dashboard/dashboard_project/database/queries.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(q, conn)


197

In [4]:
df = fetch_table("analysis_results")
df.head()

2025-08-13 17:05:37.177 No runtime found, using MemoryCacheStorageManager
/Users/nike/Documents/Data Science Work/Project/Hackthon/CX_Management_project/cx_dashboard/dashboard_project/database/queries.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,original_review_id,asin,title,region,sentiment,summary,key_drivers,urgency_score,issue_tags,primary_category,analysis_timestamp,review_date
id,,,,,,,,,,,,
1,1,B002K6AHQY,"CND Vinylux Weekly Nail Polish, Rock Royalty,...",Mumbai,Negative,The user is unhappy with the current product d...,{'Product Quality': 'Negative'},2,[quality issue],Makeup,2025-08-13 06:04:57.786863+00:00,2015-10-18 18:30:00+00:00
2,2,B00176GSEI,HOT TOOLS Professional 24k Gold Extra-Long Bar...,Chennai,Positive,The user is extremely satisfied with the curli...,"{'Longevity': 'Positive', 'Performance': 'Posi...",1,[none],Haircare,2025-08-13 06:04:57.786863+00:00,2015-11-16 18:30:00+00:00
3,3,B000ASDGK8,BaBylissPRO Ceramix Xtreme Dryer,Kolkata,Positive,"The user is satisfied with the product, statin...","{'Performance': 'Positive', 'Value for Money':...",1,[improved quality],Unspecified,2025-08-13 06:04:57.786863+00:00,2015-09-29 18:30:00+00:00
4,4,B002K6AHQY,"CND Vinylux Weekly Nail Polish, Rock Royalty,...",Kolkata,Negative,The user is disappointed with the color and fi...,"{'Finish': 'Negative', 'Color Accuracy': 'Nega...",2,"[wrong shade, wrong finish]",Fragrance,2025-08-13 06:04:57.786863+00:00,2015-07-02 18:30:00+00:00
5,5,B00FYSZDQ4,COLOR WOW Root Cover Up,Bangalore,Positive,The user is pleased with their experience usin...,"{'Finish': 'Positive', 'Ease of Application': ...",1,"[easy to use, good finish]",Makeup,2025-08-13 06:04:57.786863+00:00,2015-04-15 18:30:00+00:00


In [5]:
df['key_drivers'].str.len().value_counts()

key_drivers
1    77
2    67
3    40
4     8
0     3
6     2
Name: count, dtype: int64

In [6]:
import pandas as pd
import numpy as np

# 2. Extract positive and negative drivers into new columns
# Create a list of keys where the value is 'Positive'. If the list is empty, return np.nan
df['Positive'] = df['key_drivers'].apply(
    lambda drivers: [key for key, sentiment in drivers.items() if sentiment == 'Positive'] or np.nan
)

# Create a list of keys where the value is 'Negative'. If the list is empty, return np.nan
df['Negative'] = df['key_drivers'].apply(
    lambda drivers: [key for key, sentiment in drivers.items() if sentiment == 'Negative'] or np.nan
)


# 3. Display the final result
print("DataFrame with Positive and Negative columns:")
df.head(3)


DataFrame with Positive and Negative columns:


,original_review_id,asin,title,region,sentiment,summary,key_drivers,urgency_score,issue_tags,primary_category,analysis_timestamp,review_date,Positive,Negative
id,,,,,,,,,,,,,,
1,1,B002K6AHQY,"CND Vinylux Weekly Nail Polish, Rock Royalty,...",Mumbai,Negative,The user is unhappy with the current product d...,{'Product Quality': 'Negative'},2,[quality issue],Makeup,2025-08-13 06:04:57.786863+00:00,2015-10-18 18:30:00+00:00,NaN,[Product Quality]
2,2,B00176GSEI,HOT TOOLS Professional 24k Gold Extra-Long Bar...,Chennai,Positive,The user is extremely satisfied with the curli...,"{'Longevity': 'Positive', 'Performance': 'Posi...",1,[none],Haircare,2025-08-13 06:04:57.786863+00:00,2015-11-16 18:30:00+00:00,"[Longevity, Performance, Reliability]",NaN
3,3,B000ASDGK8,BaBylissPRO Ceramix Xtreme Dryer,Kolkata,Positive,"The user is satisfied with the product, statin...","{'Performance': 'Positive', 'Value for Money':...",1,[improved quality],Unspecified,2025-08-13 06:04:57.786863+00:00,2015-09-29 18:30:00+00:00,"[Performance, Value for Money]",NaN


In [9]:

# 3. Find the top 6 frequent drivers from the new columns
print("Top 6 Frequent Drivers:")

# For Positive drivers
try:
    # Get the value counts as a Series
    positive_counts = df['Positive'].dropna().explode().value_counts()
    
    # Get the top 6 drivers (index of the Series) and their counts
    top_6_positive = positive_counts.head(6)
    
    print("\nTop 6 Positive drivers (and their counts):")
    print(top_6_positive)

except (ValueError, IndexError):
    # This handles cases where there are no positive drivers
    print("\nNo positive drivers found or not enough data.")

# For Negative drivers
try:
    # Get the value counts as a Series
    negative_counts = df['Negative'].dropna().explode().value_counts()
    
    # Get the top 6 drivers and their counts
    top_6_negative = negative_counts.head(6)

    print("\nTop 6 Negative drivers (and their counts):")
    print(top_6_negative)

except (ValueError, IndexError):
    # This handles cases where there are no negative drivers
    print("\nNo negative drivers found or not enough data.")



Top 6 Frequent Drivers:

Top 6 Positive drivers (and their counts):
Positive
Product Performance      28
Performance              26
Product Quality          18
Value for Money          15
Ease of Use              12
Product Effectiveness     9
Name: count, dtype: int64

Top 6 Negative drivers (and their counts):
Negative
Color Accuracy         9
Product Performance    5
Application Quality    4
Product Quality        3
Product Durability     2
Value for Money        2
Name: count, dtype: int64


In [ ]:
https://medium.com/codex/streamlit-fastapi-%EF%B8%8F-the-ingredients-you-need-for-your-next-data-science-recipe-ffbeb5f76a92